In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint

# Load the UCI HAR dataset
X = pd.read_csv('/kaggle/input/ucihar-dataset/UCI-HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y = pd.read_csv('/kaggle/input/ucihar-dataset/UCI-HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None)

# Map the activity labels to human-readable names
LABELS = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']
activity_map = {1: 'Walking', 2: 'Walking Upstairs', 3: 'Walking Downstairs', 4: 'Sitting', 5: 'Standing', 6: 'Laying'}
y = y[0].map(activity_map)

# Convert human-readable labels to integer indices
le = LabelEncoder()
y = le.fit_transform(y)

# Normalize the sensor data
scaler = StandardScaler()
X = scaler.fit_transform(X.values)

# Define window size and calculate stride dynamically
window_size = 128
stride = X.shape[0] // (X.shape[0] // window_size)

# Generate sliding window samples
X_sliding = []
y_sliding = []
for i in range(0, len(X), stride):
    if i + window_size > len(X):
        break
    X_sliding.append(X[i:i+window_size])
    y_sliding.append(y[i+window_size-1])

X_sliding = np.array(X_sliding)
y_sliding = np.array(y_sliding)

# Convert activity labels to one-hot encoded vectors
y_sliding = to_categorical(y_sliding, num_classes=len(LABELS))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sliding, y_sliding, test_size=0.3, random_state=42)

# Define the model architecture
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(window_size, X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units=64))
model.add(Dropout(0.5))
model.add(Dense(units=len(LABELS), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up model checkpoint to save the best model
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1, callbacks=[checkpoint])

# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1, callbacks=[checkpoint])

# Print training loss and accuracy at each epoch
for epoch in range(1, len(history.history['loss']) + 1):
    print(f'Epoch {epoch}/{len(history.history["loss"])}')
    print(f'Training Loss: {history.history["loss"][epoch-1]:.4f} - Training Accuracy: {history.history["accuracy"][epoch-1]:.4f}')
    print(f'Validation Loss: {history.history["val_loss"][epoch-1]:.4f} - Validation Accuracy: {history.history["val_accuracy"][epoch-1]:.4f}')


Epoch 1/100
2/2 [==============================] - ETA: 0s - loss: 1.8486 - accuracy: 0.2571
Epoch 1: val_loss improved from inf to 1.65385, saving model to best_model.h5
2/2 [==============================] - 6s 2s/step - loss: 1.8486 - accuracy: 0.2571 - val_loss: 1.6539 - val_accuracy: 0.2500
Epoch 2/100
2/2 [==============================] - ETA: 0s - loss: 1.4226 - accuracy: 0.4857
Epoch 2: val_loss improved from 1.65385 to 1.52651, saving model to best_model.h5
2/2 [==============================] - 0s 132ms/step - loss: 1.4226 - accuracy: 0.4857 - val_loss: 1.5265 - val_accuracy: 0.2500
Epoch 3/100
2/2 [==============================] - ETA: 0s - loss: 1.4143 - accuracy: 0.4571
Epoch 3: val_loss improved from 1.52651 to 1.43372, saving model to best_model.h5
2/2 [==============================] - 0s 130ms/step - loss: 1.4143 - accuracy: 0.4571 - val_loss: 1.4337 - val_accuracy: 0.5000
Epoch 4/100
2/2 [==============================] - ETA: 0s - loss: 1.3449 - accuracy: 0.5429
Ep